In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 500)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import _pickle as pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from tqdm import tqdm_notebook as tqdm

In [39]:
with open("profiles.pkl",'rb') as fp:
    raw_df = pickle.load(fp)

# Loading in the clustered DF
with open("profiles_clustered.pkl",'rb') as fp:
    clguster_df = pickle.load(fp)

NameError: name 'pickle' is not defined

In [71]:
clguster_df

,Movies,Books,Music,Sports,Politics,uid,Cluster #
0,9.0,8.0,8.0,6.0,6.0,10.0,13
1,10.0,9.0,9.0,5.0,5.0,2.0,11
2,9.0,5.0,4.0,3.0,2.0,6.0,2
3,1.0,9.0,6.0,6.0,2.0,9.0,1
4,6.0,5.0,2.0,4.0,5.0,3.0,14
...,...,...,...,...,...,...,...
67,3.0,5.0,2.0,7.0,7.0,9.0,9
68,0.0,5.0,4.0,6.0,9.0,0.0,6
69,0.0,5.0,8.0,3.0,1.0,2.0,4
70,5.0,4.0,4.0,0.0,5.0,4.0,14


In [53]:
cluster_df

,Movies,Books,Music,Sports,Politics,uid,Cluster #
0,9.0,8.0,8.0,6.0,6.0,10.0,14
1,10.0,9.0,9.0,5.0,5.0,2.0,11
2,9.0,5.0,4.0,3.0,2.0,6.0,0
3,1.0,9.0,6.0,6.0,2.0,9.0,1
4,6.0,5.0,2.0,4.0,5.0,3.0,5
...,...,...,...,...,...,...,...
66,6.0,8.0,6.0,9.0,8.0,2.0,15
67,3.0,5.0,2.0,7.0,7.0,9.0,10
68,0.0,5.0,4.0,6.0,9.0,0.0,2
69,0.0,5.0,8.0,3.0,1.0,2.0,9


In [54]:
new_profile = pd.DataFrame(columns=raw_df.columns)

In [55]:
for i in new_profile.columns[0:]:
    new_profile[i] = np.random.randint(0,10,1)

In [56]:
new_profile

,Movies,Books,Music,Sports,Politics,uid
0,7,4,9,9,3,8


In [57]:
new_profile.index = [raw_df.index[-1] + 1]

In [58]:
new_cluster = raw_df.append(new_profile)

In [59]:
new_cluster

,Movies,Books,Music,Sports,Politics,uid
0,9,8,8,6,6,10
1,10,9,9,5,5,2
2,9,5,4,3,2,6
3,1,9,6,6,2,9
4,6,5,2,4,5,3
...,...,...,...,...,...,...
68,0,5,4,6,9,0
69,0,5,8,3,1,2
70,5,4,4,0,5,4
71,0,0,7,6,6,7


In [60]:
## Scaling
# Instantiating the Scaler
scaler = MinMaxScaler()

# Scaling the categories then replacing the old values
df = (pd.DataFrame(scaler.fit_transform(new_cluster), columns=new_cluster.columns[0:], index=new_cluster.index))


## Vectorizing
# Instantiating the Vectorizer
#vectorizer = CountVectorizer()

# Fitting the vectorizer to the Bios
#x = vectorizer.fit_transform(df['Bios'])

# Creating a new DF that contains the vectorized words
#df_wrds = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names())

# Concating the words DF with the original DF
#new_df = pd.concat([df, df_wrds], axis=1)

# Dropping the Bios because it is no longer needed in place of vectorization
#new_df.drop('Bios', axis=1, inplace=True)


## PCA
# Importing the library
#from sklearn.decomposition import PCA

# Instantiating PCA
#pca = PCA()

# Fitting and Transforming the DF
#df_pca = pca.fit_transform(new_df)

# Finding the exact number of features that explain at least 99% of the variance in the dataset
#total_explained_variance = pca.explained_variance_ratio_.cumsum()
#n_over_99 = len(total_explained_variance[total_explained_variance>=.99])
#n_to_reach_99 = new_df.shape[1] - n_over_99

# Reducing the dataset to the number of features determined before
#pca = PCA(n_components=n_to_reach_99)

# Fitting and transforming the dataset to the stated number of features
#df_pca = pca.fit_transform(new_df)

In [61]:
cluster_cnt = [i for i in range(2, 20, 1)]

# Establishing empty lists to store the scores for the evaluation metrics
s_scores = []

db_scores = []

# Looping through different iterations for the number of clusters
for i in tqdm(cluster_cnt):
    
    # Clustering with different number of clusters
    hac = AgglomerativeClustering(n_clusters=i)
    
    hac.fit(df)
    
    cluster_assignments = hac.labels_
    
    # Appending the scores to the empty lists    
    s_scores.append(silhouette_score(df, cluster_assignments))
    
    db_scores.append(davies_bouldin_score(df, cluster_assignments))

G:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [62]:
def cluster_eval(y, x):
    """
    Prints the scores of a set evaluation metric. Prints out the max and min values of the evaluation scores.
    """
    
    # Creating a DataFrame for returning the max and min scores for each cluster
    df = pd.DataFrame(columns=['Cluster Score'], index=[i for i in range(2, len(y)+2)])
    df['Cluster Score'] = y
    
    print('Max Value:\nCluster #', df[df['Cluster Score']==df['Cluster Score'].max()])
    print('\nMin Value:\nCluster #', df[df['Cluster Score']==df['Cluster Score'].min()])
    
    
print("\nThe Silhouette Coefficient Score (find max score):")
cluster_eval(s_scores, cluster_cnt)

print("\nThe Davies-Bouldin Score (find minimum score):")
cluster_eval(db_scores, cluster_cnt)


The Silhouette Coefficient Score (find max score):
Max Value:
Cluster #     Cluster Score
19        0.20882

Min Value:
Cluster #    Cluster Score
4       0.134595

The Davies-Bouldin Score (find minimum score):
Max Value:
Cluster #    Cluster Score
2       2.230931

Min Value:
Cluster #     Cluster Score
19        1.09171


In [63]:
## Running HAC
# Instantiating HAC
hac = AgglomerativeClustering(n_clusters=19)

# Fitting
hac.fit(df)

# Getting cluster assignments
cluster_assignments = hac.labels_

# Unscaling the categories then replacing the scaled values
df = (pd.DataFrame(scaler.inverse_transform(df), columns=df.columns[0:], index=df.index))

# Assigning the clusters to each profile
df['Cluster #'] = cluster_assignments


## Finding the Exact Cluster for our New Profile
# Getting the Cluster # for the new profile
profile_cluster = df.loc[new_profile.index]['Cluster #'].values[0]

# Using the Cluster # to narrow down the DF
profile_df = df[df['Cluster #']==profile_cluster].drop('Cluster #', axis=1)
profile_df

,Movies,Books,Music,Sports,Politics,uid
5,7.0,2.0,8.0,7.0,2.0,10.0
13,2.0,4.0,10.0,10.0,3.0,8.0
64,3.0,5.0,9.0,8.0,0.0,8.0
72,7.0,4.0,9.0,9.0,3.0,8.0


In [64]:
df

,Movies,Books,Music,Sports,Politics,uid,Cluster #
0,9.0,8.0,8.0,6.0,6.0,10.0,13
1,10.0,9.0,9.0,5.0,5.0,2.0,10
2,9.0,5.0,4.0,3.0,2.0,6.0,7
3,1.0,9.0,6.0,6.0,2.0,9.0,1
4,6.0,5.0,2.0,4.0,5.0,3.0,14
...,...,...,...,...,...,...,...
68,0.0,5.0,4.0,6.0,9.0,0.0,6
69,0.0,5.0,8.0,3.0,1.0,2.0,2
70,5.0,4.0,4.0,0.0,5.0,4.0,14
71,0.0,0.0,7.0,6.0,6.0,7.0,15


In [65]:
profile_cluster

9

In [66]:
profile_df

,Movies,Books,Music,Sports,Politics,uid
5,7.0,2.0,8.0,7.0,2.0,10.0
13,2.0,4.0,10.0,10.0,3.0,8.0
64,3.0,5.0,9.0,8.0,0.0,8.0
72,7.0,4.0,9.0,9.0,3.0,8.0


In [67]:
## Vectorizing
# Fitting the vectorizer to the Bios
#cluster_x = vectorizer.fit_transform(profile_df['Bios'])

# Creating a new DF that contains the vectorized words
#cluster_v = pd.DataFrame(cluster_x.toarray(), index=profile_df.index, columns=vectorizer.get_feature_names())

# Joining the Vectorized DF to the previous DF
#profile_df = profile_df.join(cluster_v).drop('Bios', axis=1)


## Correlation
# Trasnposing the DF so that we are correlating with the index(users) and finding the correlation
corr = profile_df.T.corr()

# Finding the Top 10 similar or correlated users to the new user
user_n = new_profile.index[0]

# Creating a DF with the Top 10 most similar profiles
top_10_sim = corr[[user_n]].sort_values(by=[user_n],axis=0, ascending=False)[1:11]

# Displaying the Top 10
raw_df.loc[top_10_sim.index]
corr

,5,13,64,72
5,1.000000,0.608424,0.711458,0.895669
13,0.608424,1.000000,0.894771,0.781568
64,0.711458,0.894771,1.000000,0.839280
72,0.895669,0.781568,0.839280,1.000000


In [19]:
new_cluster

,Movies,Books,Music,Sports,Politics,uid
0,9,8,8,6,6,10
1,10,9,9,5,5,2
2,9,5,4,3,2,6
3,1,9,6,6,2,9
4,6,5,2,4,5,3
...,...,...,...,...,...,...
66,6,8,6,9,8,2
67,3,5,2,7,7,9
68,0,5,4,6,9,0
69,0,5,8,3,1,2


In [50]:
with open("profiles.pkl","wb") as fp:
    pickle.dump(new_cluster,fp)

with open("profiles_clustered.pkl","wb") as fp:
    pickle.dump(df,fp)

In [72]:
df

,Movies,Books,Music,Sports,Politics,uid,Cluster #
0,9.0,8.0,8.0,6.0,6.0,10.0,13
1,10.0,9.0,9.0,5.0,5.0,2.0,10
2,9.0,5.0,4.0,3.0,2.0,6.0,7
3,1.0,9.0,6.0,6.0,2.0,9.0,1
4,6.0,5.0,2.0,4.0,5.0,3.0,14
...,...,...,...,...,...,...,...
68,0.0,5.0,4.0,6.0,9.0,0.0,6
69,0.0,5.0,8.0,3.0,1.0,2.0,2
70,5.0,4.0,4.0,0.0,5.0,4.0,14
71,0.0,0.0,7.0,6.0,6.0,7.0,15


In [105]:
dee = dee.drop('Books', axis=1)

In [106]:
dee

,uid
0,10.0
1,2.0
2,6.0
3,9.0
4,3.0
...,...
68,0.0
69,2.0
70,4.0
71,7.0


In [107]:
with open("profilesMapIndex.pkl","wb") as fp:
    pickle.dump(dee,fp)

In [108]:
np.random.randint(0,10,1)

array([2])

In [26]:
fs = []
topic_name = []

In [27]:
t=('Sport', {'baseball': '0', 'basketball': '0', 'chess': '0', 'cricket': '1', 'football': '1', 'hockey': '0', 'value': '4'})

In [28]:
for i in t[1].items():
    fs.append(i[1])
    topic_name.append(i[0])


In [29]:
fs

['0', '0', '0', '1', '1', '0', '4']

In [38]:
len(topic_name)

7

In [31]:
topic_name.pop()

'value'

In [33]:
topic_name.append(t[0])

In [40]:
temp_list = ['0', '0', '0', '0', '1', '1', '9', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '9', '0', '1', '1', '0', '0', '0', '1', '9', '0', '0', '0', '1', '1', '0', '4']
len(temp_list)

33

In [41]:
temp_list

['0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '9',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '9',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '9',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '4']

In [46]:
for j in range(0,33):
    print((int(temp_list[j])))

0
0
0
0
1
1
9
0
0
1
0
0
0
0
0
1
0
9
0
1
1
0
0
0
1
9
0
0
0
1
1
0
4
